In [5]:
import pandas as pd
import numpy as np
import os
import warnings
import time

import json

from skrub import TableReport
from ollama import Client
from pdfquery import PDFQuery
from pprint import pprint

import bid_utils

In [6]:
## load the dfs built in 1_collect_files
rep_drive_ATAE =  r"C:\Users\jch_m\ATAE"
rep_data =        r"C:\DocPerso\Dev_Python\Data\ATAE"
rep_data_output = r"C:\DocPerso\Dev_Python\ProjetsPerso\BidManagement\data_output"

path_df_EBP = os.path.join(rep_data_output, "df_EBP.pkl")
df_EBP = pd.read_pickle(path_df_EBP)

path_df_consult = os.path.join(rep_data_output, "df_consult.pkl")
df_consult = pd.read_pickle(path_df_consult)

path_df_rejet = os.path.join(rep_data_output, "df_rejet.pkl")
df_rejet = pd.read_pickle(path_df_rejet)


In [7]:
# Merge df_consult and df_EBP pour sync EBP ID on the mission/files
# note: no need to sync EBP with rejet, because none of the Rejet file match an EBP entry (dans la liste des repertoires et fichiers). normal ?

# Define the columns to match on
match_columns = ['SPS Name', 'Ville', 'Entreprise', 'Mission']

df_consult_ebp = pd.merge(df_consult, df_EBP, on=match_columns, how='left', suffixes=('_consult', '_ebp'))
df_consult_ebp['ID EBP'] = df_consult_ebp['ID EBP_ebp'].fillna("no EBP")
df_consult_ebp = df_consult_ebp.drop(columns=['ID EBP_ebp','ID EBP_consult', 'statut_ebp'])

df_consult_ebp['file_name'] = df_consult_ebp['file_path'].str.split(r'\\').str[-1]

In [8]:
# define a unique 'no EBP xx'  for each combinaison of SPS+Ville+Entreprise+Mission, for all related files
#
mask = df_consult_ebp["ID EBP"] == "no EBP"

# Create the combined series only for the masked rows
combined_series_for_update = (
    df_consult_ebp.loc[mask, 'SPS Name'].astype(str) + '_' +
    df_consult_ebp.loc[mask, 'Ville'].astype(str) + '_' +
    df_consult_ebp.loc[mask, 'Entreprise'].astype(str) + '_' +
    df_consult_ebp.loc[mask, 'Mission'].astype(str)
)

# Generate unique IDs for this series
unique_ids, _ = pd.factorize(combined_series_for_update)

# Assign back to the original DataFrame using the mask
df_consult_ebp.loc[mask, "ID EBP"] = "no EBP " + (unique_ids + 1).astype(str)


In [9]:
# Add columns "AO_docs" (True/False), "AO_Doc_type"(CCTP, CCAP, RC, AAPC, Memo_tech), "Commande"(True/False)

df_consult_ebp['AO_docs'] = (
    (df_consult_ebp['file_path'].str.lower().str.contains('devis') & df_consult_ebp['file_path'].str.lower().str.contains(r'\\consul')) | 
    (df_consult_ebp['file_path'].str.lower().str.contains(r'\\adm') & df_consult_ebp['file_path'].str.lower().str.contains(r'\\consul'))
)


# Set AO_doc_type
# Initialize all with default value
df_consult_ebp['AO_doc_type'] = 'no type'
mask = df_consult_ebp['AO_docs'] == True
df_consult_ebp.loc[mask & df_consult_ebp['file_name'].str.contains('CCTP', case=False, na=False), 'AO_doc_type'] = 'CCTP'
df_consult_ebp.loc[mask & df_consult_ebp['file_name'].str.contains('CCP', case=False, na=False), 'AO_doc_type'] = 'CCTP'
df_consult_ebp.loc[mask & df_consult_ebp['file_name'].str.contains('Lettre', case=False, na=False), 'AO_doc_type'] = 'Lettre Consult'
df_consult_ebp.loc[mask & df_consult_ebp['file_name'].str.contains('courier', case=False, na=False), 'AO_doc_type'] = 'Lettre Consult'
df_consult_ebp.loc[mask & df_consult_ebp['file_name'].str.contains('consult', case=False, na=False), 'AO_doc_type'] = 'Lettre Consult'
df_consult_ebp.loc[mask & df_consult_ebp['file_name'].str.contains('CCAP', case=False, na=False), 'AO_doc_type'] = 'CCAP'
df_consult_ebp.loc[mask & df_consult_ebp['file_name'].str.contains('AAPC', case=False, na=False), 'AO_doc_type'] = 'AAPC'
df_consult_ebp.loc[mask & df_consult_ebp['file_name'].str.contains('RC', case=False, na=False), 'AO_doc_type'] = 'RC'
df_consult_ebp.loc[mask & df_consult_ebp['file_name'].str.contains('glement', case=False, na=False), 'AO_doc_type'] = 'RC'
df_consult_ebp.loc[mask & df_consult_ebp['file_name'].str.contains('planning', case=False, na=False), 'AO_doc_type'] = 'Planning'
df_consult_ebp.loc[mask & df_consult_ebp['file_name'].str.contains('phasag', case=False, na=False), 'AO_doc_type'] = 'Planning'

In [93]:
list_file_AO_notype = ""
mask2 = (df_consult_ebp['AO_docs'] == True) &  (df_consult_ebp['AO_doc_type'] == 'no type')
list_file_AO_notype = df_consult_ebp.loc[mask2, 'file_name']
for file in list_file_AO_notype:
    if "plan " in file.lower() or "plans" in file.lower() or "coupe" in file.lower() or "vue" in file.lower() or "facade" in file.lower() or "lot" in file.lower():
        continue
    print(file)

Estimation ESQ-DIAG Modulaire Pierre et Marie Curie.pdf

CSPS Contexte_Filières_HLX.pdf

Heinlex Filieres-Programme-rev2.pdf

22062-PA8b-2023-06-22.pdf

2023-12-12-A1_existant.pdf

2023-12-12-A1_projet.pdf

M22.054-CR01-20230907.pdf

M22054-21-voirie.pdf

Fiche identité CSPS.pdf

1 DCE CSPS_230321.pdf

Attestation 2021- 52-301 - SARL ATAE.pdf

1 DCE CSPS_230321.pdf

2-Note méthodologique VENDEE EAU.pdf

3 DE SPS_230321 6 sites.pdf

3 DE SPS_230321 RECAPITULATIF.pdf

4-Attestation sur l'Honneur.pdf

5-Attestation Assurance 2021 ATAE.pdf

6-DC1.pdf

7-DC2 et annexes.pdf

Programme technique.pdf

ANNEXE tableau financier TVX base.pdf

ANNEXE tableau financier TVX options.pdf

AVP TOITURE CHLVO.pdf

03b DTA _  _ Travaux toiture Lege.pdf

04 Mémoire technique.pdf

Mémoire technique Blandin Travaux toiture Lege.pdf

Cahier charges mission CSPS_Boucardière - ATAE.pdf

Cahier charges mission CSPS_Boucardière.pdf

MACHECOUL - MASSE - 2017 02 28.pdf

MACHECOUL - MASSE - 2017 02 28.pdf

00-Courri

In [95]:
df_consult_ebp['AO_docs'].value_counts()

AO_docs
False    152162
True       3095
Name: count, dtype: int64

In [85]:
df_consult_ebp['AO_doc_type'].value_counts()

AO_doc_type
no type           153975
CCTP                 469
RC                   457
CCAP                 200
Planning              92
Lettre Consult        34
AAPC                  30
Name: count, dtype: int64

In [71]:
df_consult_ebp.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155257 entries, 0 to 155256
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   SPS Name        155257 non-null  object
 1   Ville           155257 non-null  object
 2   Entreprise      155257 non-null  object
 3   Mission         155257 non-null  object
 4   statut_consult  155257 non-null  object
 5   file_path       155257 non-null  object
 6   ID EBP          155257 non-null  object
 7   file_name       155257 non-null  object
 8   AO_docs         155257 non-null  bool  
 9   AO_doc_type     155257 non-null  object
dtypes: bool(1), object(9)
memory usage: 10.8+ MB


In [10]:
TableReport(df_consult_ebp)

Processing column  10 / 10


,,,,,,,,,,


In [11]:
# Creation d'un DF_consult_elevated avec une ligne par "EBP ID", 
# et l'ajout des features resultants de l'extraction 

df_consult_ao = df_consult_ebp[df_consult_ebp['AO_docs']==True]
df_consult_elevated = df_consult_ao.drop_duplicates(subset=["ID EBP"], keep="first")

# Suppression des colonnes desormais inutiles
df_consult_elevated = df_consult_elevated.drop(columns=['file_path','file_name','AO_docs','AO_doc_type'])

df_consult_elevated['lieu'] = ""
df_consult_elevated['type travaux'] = ""
df_consult_elevated['phase mission'] = ""
df_consult_elevated['duree travaux'] = 0
df_consult_elevated['prix travaux'] = 0.0


In [12]:
df_consult_elevated.info()


<class 'pandas.core.frame.DataFrame'>
Index: 923 entries, 0 to 155112
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   SPS Name        923 non-null    object 
 1   Ville           923 non-null    object 
 2   Entreprise      923 non-null    object 
 3   Mission         923 non-null    object 
 4   statut_consult  923 non-null    object 
 5   ID EBP          923 non-null    object 
 6   lieu            923 non-null    object 
 7   type travaux    923 non-null    object 
 8   phase mission   923 non-null    object 
 9   duree travaux   923 non-null    int64  
 10  prix travaux    923 non-null    float64
dtypes: float64(1), int64(1), object(9)
memory usage: 86.5+ KB


# Test de modele pour resumer et extraire l'info un CCTP 

In [13]:
# load file 1 consult file for example

file_name = ""
if (df_consult_ebp['statut_consult'] == "Chantier").any():
    first_index_idxmax = (df_consult_ebp['statut_consult'] == "Chantier").idxmax()
    print(f"\nPremier index avec 'statut_consult' == 'Chantier' (idxmax) : {first_index_idxmax}")
    file_name = df_consult_ebp['file_path'][first_index_idxmax].strip()
    print(f"File Name :     {file_name}")
else:
    print("\nAucune ligne trouvée avec 'statut_consult' == 'Chantiers'.")

warnings.filterwarnings("ignore")

pdf = PDFQuery(file_name)
pdf.load()

# Use CSS-like selectors to locate the elements
text_elements = pdf.pq('LTTextLineHorizontal')

# Extract the text from the elements
text = [t.text for t in text_elements if t.text != '' or t.text != ' ']

#print(text)



Premier index avec 'statut_consult' == 'Chantier' (idxmax) : 86
File Name :     C:\Users\jch_m\ATAE\Cyrille CHARTIER - CYRILLE\01-CHANTIERS\00-NOUVEAU\BERNERIE - LAD - PRE BOISMAIN\ADMINISTRATIF\Consult\CCTP_CSPS_Tranche 2 définitive.pdf


In [14]:
# Init Ollama, with list of model, prompt

ollama_url = "http://localhost:11434"

# 'mistral-small3.1' #14G
list_model = ["gemma3:4b", "llama3.2", "minicpm-v", "mistral-small3.1"]
# Get an ollama client
llmclient = Client(host=ollama_url)

model_options = {
    "num_predict": 1300,  # max number of tokens to predict
    "temperature": 0.1,
    "top_p": 0.9,
}



In [148]:
"""
prompt_task = "En Francais, résume ce texte et extrait les points importants de la mission et les caractéristiques du contrat (prix, durée, lieu, nom du maitre d'oeuvre, nom du lieu). Donne uniquement les faits et informations disponible de text , sans interpretation ou estimation. Donne des reponses chiffrées et quantifiées lors que c'est possible. La reponse doit avoir un maximum 900 mots\n"

prompt_full = prompt_task + "".join(text)

# Loop on the list of models specified
for i in range(len(list_model)):
    model_name = list_model[i]
    print("\n==========================================")
    print(f"===== Test model :{i+1}/{len(list_model)}: {model_name} =========\n")
    time_start = time.time()
    result = llmclient.generate(
        model=model_name, prompt=prompt_full, options=model_options
    )
    print(f"\nDuree execution ={time.time()-time_start:.0f}secondes")

    pprint(result, compact=False, sort_dicts=False)
    utils.print_text_wrapped("\nLLM response:\n" + result["response"])

"""

'\nprompt_task = "En Francais, résume ce texte et extrait les points importants de la mission et les caractéristiques du contrat (prix, durée, lieu, nom du maitre d\'oeuvre, nom du lieu). Donne uniquement les faits et informations disponible de text , sans interpretation ou estimation. Donne des reponses chiffrées et quantifiées lors que c\'est possible. La reponse doit avoir un maximum 900 mots\n"\n\nprompt_full = prompt_task + "".join(text)\n\n# Loop on the list of models specified\nfor i in range(len(list_model)):\n    model_name = list_model[i]\n    print("\n==========================================")\n    print(f"===== Test model :{i+1}/{len(list_model)}: {model_name} =========\n")\n    time_start = time.time()\n    result = llmclient.generate(\n        model=model_name, prompt=prompt_full, options=model_options\n    )\n    print(f"\nDuree execution ={time.time()-time_start:.0f}secondes")\n\n    pprint(result, compact=False, sort_dicts=False)\n    utils.print_text_wrapped("\nLLM 

In [ ]:
# pour 1 modele, produit un json 
# prompt_task = "summarize in french this text and highlight key points of the mission and bid features to consider in less than 900 words\n"
prompt_task_resumer = "En Francais, extrait de ce texte les points suivants : \
   'Nom Chantier' : \
   'Lieu du Chantier (Commune, Departement, Rue)' : \
   'Maitre ouvrage': , \
   'Maitre oeuvre': , \
   'Type de travaux': \
   'Phases de la mission' : \
   'Prix des travaux (en euros)': \
   'Duree Previsionnelle des Travaux (en mois)': \
   'Categorie operation SPS' : \
Donne uniquement les faits et informations disponible de text , sans interpretation ou estimation. \
Donne des reponses chiffrées et quantifiées lors que c'est possible. \
La reponse doit avoir un maximum 900 mots"

prompt_task_json = "Formate les informations au format JSON , en respectant strictement les clefs et types specifies tels que \n\
{ \
    'Nom Chantier': '<string>', \
    'Lieu du Chantier': '<string>', \
    'Maitre d'ouvrage': '<string>', \
    'Maitre d'oeuvre': '<string>', \
    'Type de Travaux': '<string>', \
    'Phase de la mission': '<string>', \
    'Durée Prévisionnelle des Travaux (en mois)': '<integer>', \
    'Prix des travaux (en euros HT)': '<float>', \
    'Categorie operation SPS': '<string>' \
} \
Donne des reponses uniquement pour ces clefs. en chiffre lorsque c'est possible. Si pas d'info, laisser vide\n"

model_name = "gemma3:4b"
#Resumer
prompt_full = prompt_task_resumer + "".join(text)
result = llmclient.generate( model=model_name, prompt=prompt_full, options=model_options)
#pprint(result, compact=True, sort_dicts=False)
#utils.print_text_wrapped("\nLLM response:\n" + result["response"])

#JSON
prompt_full = prompt_task_json + "".join(result["response"])
result = llmclient.generate( model=model_name, prompt=prompt_full, options=model_options)
#pprint(result, compact=True, sort_dicts=False)
#utils.print_text_wrapped("\nLLM response:\n" + result["response"])

bid_utils.print_json_info_cctp(result["response"])

AttributeError: module 'bid_utils' has no attribute 'print_json_info_chantier'

In [ ]:
"""
model_name = "mistral-small3.1"

#Resumé
prompt_full = prompt_task_resumer + "".join(text)
result = llmclient.generate( model=model_name, prompt=prompt_full, options=model_options)
pprint(result, compact=True, sort_dicts=False)
bid_utils.print_text_wrapped("\nLLM response:\n" + result["response"])

#JSON
prompt_full = prompt_task_json + "".join(result["response"])
result = llmclient.generate( model=model_name, prompt=prompt_full, options=model_options)
pprint(result, compact=True, sort_dicts=False)
bid_utils.print_text_wrapped("\nLLM response:\n" + result["response"])

bid_utils.print_json_info_chantier(result["response"])
"""

GenerateResponse(model='mistral-small3.1', created_at='2025-06-08T17:29:19.1352817Z', done=True, done_reason='stop', total_duration=593031959500, load_duration=10297736700, prompt_eval_count=4096, prompt_eval_duration=334491705900, eval_count=911, eval_duration=248240221200, response="Le document que vous avez fourni est un cahier des charges détaillé pour le lot électrique d'un projet de réaménagement et d'extension d'un local commercial. Voici un résumé des points clés et des exigences principales :\n\n### 1. **Préambule et Contexte**\n- **Projet** : Réaménagement et extension d'un local commercial.\n- **Localisation** : VLOK 80000 Mouilleron-le-Captif.\n- **Objectif** : Définir les prestations attendues pour le lot électrique.\n\n### 2. **Prestations Générales**\n- **Accessoires de fixation et raccordement** : Fourniture et pose.\n- **Percements, rebouchages, coupe-feu** : Réalisation nécessaire.\n- **Moulures, goulottes, tubes, gaines** : Fourniture et pose.\n- **Matériel de levage

In [ ]:
## Liste all files for given consultation
## Consolide les fichiers bruts ou les resumés
## Run le modele pour resumer et json

# Preferable de faire une extraction par type de fichier (CCTP, CCAP, ..) avec un prompt dédié.
# donc regroupement par type de fichier

import numpy as np

model_name = "gemma3:4b"
nb_of_projet_to_test = 5
count_id = 1

# liste des ID correspondants au mask
condition_mask = (df_consult_ebp['AO_docs'] == True) & \
                    (df_consult_ebp['AO_doc_type'] == "CCTP") 

matching_ebp_ids = df_consult_ebp.loc[condition_mask, 'ID EBP'].unique()

if len(matching_ebp_ids) > 0:
    # Determine how many unique IDs we can actually select (min of 5 or available count)
    num_to_select = min(nb_of_projet_to_test, len(matching_ebp_ids))

    # Randomly select the desired number of unique EBP IDs
    selected_ebp_ids = np.random.choice(matching_ebp_ids, num_to_select, replace=False)

    print(f"\n{num_to_select} ID(s) EBP aléatoire(s) sélectionné(s) correspondant au filtre:")
    for ebp_id in selected_ebp_ids:

        filtered_row = df_consult_ebp[df_consult_ebp['ID EBP'] == ebp_id].iloc[0]
        print(f"\n{count_id}/{nb_of_projet_to_test} => ID:{ebp_id} statut_consult: {filtered_row['statut_consult']}")

        text_multi_files = ""
        list_file_consult = df_consult_ebp[(df_consult_ebp['ID EBP'] == ebp_id) & (df_consult_ebp['AO_doc_type'] == 'CCTP')]['file_path']
        print(f"Il y a {len(list_file_consult)} fichiers dans la dossier AO consultation pour ce projet {ebp_id}")
        print(f"Ville:{filtered_row['Ville']}  Entreprise:{filtered_row['Entreprise']}  Mission:{filtered_row['Mission']}")
        for file_path in list_file_consult:
            #if "\\adm" in file_path.lower() and "\\consul" in file_path.lower():
                nb_segment = file_path.split("\\")
                print("==> ", nb_segment[-1].strip())
                pdf = PDFQuery(file_path.strip())
                pdf.load()
                text_elements = pdf.pq('LTTextLineHorizontal')
                text = [t.text for t in text_elements if t.text != '' or t.text != ' ']
                text_multi_files += " ".join(text) + "\n" # Contatenate text
            #else:
                #print("not consult --:", file_path.split("\\")[-1].strip())
            #    pass

        if text_multi_files != "":

            print("Run model for multiple files")

            #Resumer
            prompt_full = prompt_task_resumer + "".join(text_multi_files)
            result = llmclient.generate( model=model_name, prompt=prompt_full, options=model_options)
            #pprint(result, compact=True, sort_dicts=False)
            #bid_utils.print_text_wrapped("\nLLM response:\n" + result["response"])

            #JSON
            prompt_full = prompt_task_json + "".join(result["response"])
            result = llmclient.generate( model=model_name, prompt=prompt_full, options=model_options)
            #pprint(result, compact=True, sort_dicts=False)
            #bid_utils.print_text_wrapped("\nLLM response:\n" + result["response"])

            cctp_json = bid_utils.print_json_info_cctp(result["response"])
            
            if cctp_json != "":
                parsed_json = {} # Initialize an empty dictionary
                try:
                    parsed_json = json.loads(cctp_json)
                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON string: {e}")
                except TypeError:
                    print("Input is not a string.") 
                
                lieu_value = parsed_json.get("Lieu du Chantier")
                type_travaux_value = parsed_json.get("Type de Travaux")
                phase_mission_value = parsed_json.get("Phase de la mission")
                duree_travaux_value = parsed_json.get("Durée Prévisionnelle des Travaux (en mois)")
                prix_travaux_value = parsed_json.get("Prix des travaux (en euros HT)")
   
                mask = df_consult_elevated['ID EBP'] == ebp_id

                if lieu_value is not None:
                    df_consult_elevated.loc[mask, 'lieu'] = lieu_value

                if type_travaux_value is not None:
                    df_consult_elevated.loc[mask, 'type travaux'] = type_travaux_value
                
                if duree_travaux_value is not None:
                    df_consult_elevated.loc[mask, 'lieu'] = duree_travaux_value

                if phase_mission_value is not None:
                    df_consult_elevated.loc[mask, 'phase mission'] = phase_mission_value
                
                if prix_travaux_value is not None:
                    df_consult_elevated.loc[mask, 'prix travaux'] = prix_travaux_value

        else:
            print("Pas de fichiers consult à processer")
        
        count_id += 1
else:
    print("\nAucune ligne trouvée avec 'statut_consult' == 'Chantiers'")





5 ID(s) EBP aléatoire(s) sélectionné(s) correspondant au filtre:

ID:220292 statut_consult: Chantier
Il y a 1 fichiers dans la dossier AO consultation pour ce projet 220292
Ville:ST CHRISTOPHE DU LIGNERON  Entreprise:VE & MAIRIE  Mission:La Brosse
==>  615_CCTP_COMMUN_v1.pdf
Run model for multiple files
JSON extrait et chargé dans un dictionnaire Python :
{'Nom Chantier': '', 'Lieu du Chantier': 'Non spécifié', "Maitre d'ouvrage": 'Non spécifié', "Maitre d'oeuvre": 'Non spécifié', 'Type de Travaux': 'Non spécifié', 'Phase de la mission': 'Non spécifié', 'Durée Prévisionnelle des Travaux (en mois)': 'Non spécifié', 'Prix des travaux (en euros HT)': 'Non spécifié', 'Categorie operation SPS': 'Non spécifié'}

Type de l'objet : <class 'dict'>

Accès aux données du dictionnaire :
Nom Chantier: 
Lieu du Chantier: Non spécifié
Maitre ouvrage: Information non disponible
Maitre oeuvre: Information non disponible
Type de Travaux: Non spécifié
Phase de la mission: Non spécifié
Durée Prévisionnel

Cannot set gray non-stroke color because /'R63' is an invalid float value
Cannot set gray non-stroke color because /'R63' is an invalid float value
Cannot set gray non-stroke color because /'R63' is an invalid float value
Cannot set gray non-stroke color because /'R55' is an invalid float value
Cannot set gray non-stroke color because /'R148' is an invalid float value


==>  968 - Lot 16 CCTP Plomberie Sanitaires.pdf
==>  968 - Lot 17 CCTP Chauffage Ventilation.pdf
==>  968 - Lot 18 CCTP.pdf
==>  214079 CCTP LOT EQUIPEMENTS DE CUISINE + cuisine provisoire 2015-09-02.pdf


# Etude des Rejets

In [ ]:
print(df_rejet['SPS Name'][20])
print(df_rejet['Directory'][20])
print(df_rejet['filename'][20])

file_path = os.path.join(path_root, df_rejet['SPS Name'][20], df_rejet['Directory'][20], df_rejet['filename'][20])
print(file_path)
print(os.path.exists(file_path))

In [ ]:
list_fichier_rejet_valid = []

for i in range(len(df_rejet)):
    file_path = os.path.join(path_root, df_rejet['SPS Name'][i], df_rejet['Directory'][i], df_rejet['filename'][i])
    if os.path.exists(file_path):
        try: # certains nom de fichiers provoquent des erreurs
            pdf = PDFQuery(file_path)
            pdf.load()

            # Attention les documents scannés (image) ne chargent aucun texte avec PDFQuery

            # Use CSS-like selectors to locate the elements
            text_elements = pdf.pq('LTTextLineHorizontal')
            if len(text_elements) > 20:
                #print(f"Found {len(text_elements)} LTTextLineHorizontal elements.")

                # Extract the text from the elements
                text = [t.text for t in text_elements if t.text != '' or t.text != ' ']
                list_fichier_rejet_valid.append(i)
            else:
                print(i, "=== Pas de text ", file_path )
                pass
        except Exception as e:
            print(f"*** Error processing {file_path}: {e}")        
    else:
        print(i, "=== Existe pas ", file_path )
        pass


In [ ]:
print("Nb de fichiers de rejet valides :",len(list_fichier_rejet_valid), " sur ", len(df_rejet), "fichiers")
print(list_fichier_rejet_valid)

#file_path = os.path.join(path_root, df_rejet['SPS Name'][104], df_rejet['Directory'][104], df_rejet['filename'][104])
item_valid_rejet_file = np.random.choice(list_fichier_rejet_valid, 1)[0]
file_path = os.path.join(path_root, df_rejet['SPS Name'][item_valid_rejet_file], df_rejet['Directory'][item_valid_rejet_file], df_rejet['filename'][item_valid_rejet_file])
print(file_path)

In [ ]:
# Creer une liste des documents de rejet "valide" (fichier exist et contient du text)
#
item_valid_rejet_file = np.random.choice(list_fichier_rejet_valid, 1)[0]
file_path = os.path.join(path_root, df_rejet['SPS Name'][item_valid_rejet_file], df_rejet['Directory'][item_valid_rejet_file], df_rejet['filename'][item_valid_rejet_file])
if os.path.exists(file_path):
    print("Fichier choisi :",file_path)
    pdf = PDFQuery(file_path)
    pdf.load()
    # Attention les documents scannés (image) ne chargent aucun texte aavec PDFQuery

    # Use CSS-like selectors to locate the elements
    text_elements = pdf.pq('LTTextLineHorizontal')
    if len(text_elements) > 20:
        #print(f"Found {len(text_elements)} LTTextLineHorizontal elements.")

        # Extract the text from the elements
        text = [t.text for t in text_elements if t.text != '' or t.text != ' ']
    else:
        print("ERREUR : pas de text dans le pdf")
else:
    print("ERREUR: le fichier n'est pas accessible ")


In [ ]:
# Execution du modele pour 3 exemples de fichiers "rejet" ayant du text_elements
model_name = 'llama3.2:latest'

model_options = {
    'num_predict': 1300,  # max number of tokens to predict
    'temperature': 0.1,
    'top_p': 0.9,
}

prompt_task = "summarize this text in french and highlight the reason for the ATAE bid failure, the motivation for the decision , the bid winner name and the price awarded in less than 500 words\n"

# test 3 fichiers parmi les valides
for i in range(3):
    print("\n====== Test ", i+1)
    item_valid_rejet_file = np.random.choice(list_fichier_rejet_valid, 1)[0]
    file_path = os.path.join(path_root, df_rejet['SPS Name'][item_valid_rejet_file], df_rejet['Directory'][item_valid_rejet_file], df_rejet['filename'][item_valid_rejet_file])
    if os.path.exists(file_path):
        print("Fichier choisi :", item_valid_rejet_file," : ",file_path)
        pdf = PDFQuery(file_path)
        pdf.load()
        # Attention les documents scannés (image) ne chargent aucun texte aavec PDFQuery

        # Use CSS-like selectors to locate the elements
        text_elements = pdf.pq('LTTextLineHorizontal')
        if len(text_elements) > 20:
            #print(f"Found {len(text_elements)} LTTextLineHorizontal elements.")

            # Extract the text from the elements
            text = [t.text for t in text_elements if t.text != '' or t.text != ' ']
        else:
            print("ERREUR : pas de text dans le pdf")
            break
    else:
        print("ERREUR: le fichier n'est pas accessible ")
        break
        
    prompt_full = prompt_task + "".join(text)

    result = llmclient.generate(model=model_name, prompt=prompt_full, options=model_options)

    pprint(result, compact=False, sort_dicts=False)

    #print("LLM response:\n", result['response'])
    bid_utils.print_text_wrapped("\nLLM response:\n" + result['response'])

In [ ]:
item_valid_rejet_file = 1 #55 94
file_path = os.path.join(path_root, df_rejet['SPS Name'][item_valid_rejet_file], df_rejet['Directory'][item_valid_rejet_file], df_rejet['filename'][item_valid_rejet_file])
if os.path.exists(file_path):
    print("Fichier choisi :", item_valid_rejet_file," : ",file_path)
    pdf = PDFQuery(file_path)
    pdf.load()
    # Attention les documents scannés (image) ne chargent aucun texte aavec PDFQuery

    # Use CSS-like selectors to locate the elements
    text_elements = pdf.pq('LTTextLineHorizontal')
    if len(text_elements) > 20:
        #print(f"Found {len(text_elements)} LTTextLineHorizontal elements.")

        # Extract the text from the elements
        text = [t.text for t in text_elements if t.text != '' or t.text != ' ']
    else:
        print("ERREUR : pas de text dans le pdf")
        
else:
    print("ERREUR: le fichier n'est pas accessible ")
    
    
prompt_full = prompt_task + "".join(text)

result = llmclient.generate(model=model_name, prompt=prompt_full, options=model_options)

pprint(result, compact=False, sort_dicts=False)

print("LLM response:\n", result['response'])